In [17]:
import pandas as pd
import re
import json
from time import sleep 
from random import randint
from selenium import webdriver
# driver = webdriver.Chrome('H:\Digital_Media\climate_change\climate-chatbot\chromedriver_win32\chromedriver.exe') 

In [ ]:
for url in open("list_of_urls.txt").readlines():
    driver.get(url)
    linkLists = driver.find_elements_by_class_name("summary h3 a")
    for link in linkLists: 
        print(link.get_attribute('href'))
        driver.get(link.get_attribute('href'))
        break
driver.close()

In [18]:
def getDateTime(dateTimeString):
    return dateTimeString.split(" ")[0]

In [36]:
driver = webdriver.Chrome('H:\Digital_Media\climate_change\climate-chatbot\chromedriver_win32\chromedriver.exe') 
mainJson = {}
for url in open("list_of_urls.txt").readlines():
    driver.get(url)
    
    # Question Header Div
    questionHeader = driver.find_element_by_id("question-header")
    mainJson["question"] = questionHeader.find_element_by_tag_name("h1").text
    
    # Question Divs
    question = driver.find_element_by_class_name("question")
    questionId = int(question.get_attribute("data-questionid"))
    mainJson["author"] = question.find_element_by_xpath("//div[@class='user-details']/span[@class='d-none']").get_attribute("innerHTML")
    mainJson["date"] = getDateTime(question.find_element_by_xpath("//div[@class='user-action-time']/span[@class='relativetime']").get_attribute("title"))
    mainJson["text"] = question.find_element_by_class_name("post-text").text
    
    # Question Comments Div
    mainJson["comments"] = []
    comments = question.find_element_by_xpath("//div[@class='post-layout--right']/div[@id='comments-%d']" % questionId)
    commentDiv = question.find_element_by_id("comments-%d" % questionId)
    ulCommentDiv = commentDiv.find_element_by_class_name("comments-list")    
    liCommentDiv = ulCommentDiv.find_elements_by_class_name("comment")
    for li in liCommentDiv:
        commentJson = {}
        commentJson["author"] = li.find_element_by_class_name("comment-user").text
        commentJson["date"] = getDateTime(li.find_element_by_class_name("relativetime-clean").get_attribute("title"))
        commentJson["text"] = li.find_element_by_class_name("comment-copy").text
        mainJson["comments"].append(commentJson)
    

    # Answers Div
    mainJson["answers"] = []
    answersDiv = driver.find_element_by_id("answers")
    answers = answersDiv.find_elements_by_class_name("answer")
    for answer in answers:
        answerJson = {}
        answerId = int(answer.get_attribute("data-answerid"))
        userInfos = answer.find_elements_by_class_name("user-info")
        for userInfo in userInfos:
            userActionTime = userInfo.find_element_by_class_name("user-action-time")
            innerHTML = userActionTime.get_attribute("innerHTML")
            pattern = re.compile(r'answered', re.I)
            if pattern.search(innerHTML) is not None:
                answerJson["author"] = userInfo.find_element_by_class_name("d-none").get_attribute("innerHTML")
                answerJson["date"] = getDateTime(userInfo.find_element_by_class_name("relativetime").get_attribute("title"))
        answerJson["text"] = answer.find_element_by_class_name("post-text").text
                
        # Anser Comments Div
        answerJson["comments"] = []
        commentsJson = {}
        comments = answer.find_element_by_xpath("//div[@class='post-layout--right']/div[@id='comments-%d']" % answerId)
        commentDiv = answer.find_element_by_id("comments-%d" % answerId)
        ulCommentDiv = commentDiv.find_element_by_class_name("comments-list")
        
        # Check if some comments are not expanded 
        dataRemainingCommentsCount = int(ulCommentDiv.get_attribute("data-remaining-comments-count"))
        if(dataRemainingCommentsCount != 0):
            commentsLink = answer.find_element_by_xpath("//div[@class='post-layout--right']/div[@id='comments-link-%d']" % answerId)
            element = commentsLink.find_element_by_partial_link_text('show')
            driver.execute_script("arguments[0].click();", element)
            sleep(1) # seconds
            liCommentDiv = ulCommentDiv.find_elements_by_class_name("comment")
            for li in liCommentDiv:
                commentJson["author"] = li.find_element_by_class_name("comment-user").text
                commentJson["date"] = getDateTime(li.find_element_by_class_name("relativetime-clean").get_attribute("title"))
                commentJson["text"] = li.find_element_by_class_name("comment-copy").text
                answerJson["comments"].append(commentJson)
        else:
            liCommentDiv = ulCommentDiv.find_elements_by_class_name("comment")
            for li in liCommentDiv:
                commentJson["author"] = li.find_element_by_class_name("comment-user").text
                commentJson["date"] = getDateTime(li.find_element_by_class_name("relativetime-clean").get_attribute("title"))
                commentJson["text"] = li.find_element_by_class_name("comment-copy").text
                answerJson["comments"].append(commentJson)
        mainJson["answers"].append(answerJson)
s = json.dumps(mainJson)
with open("myJson.json", "w") as f:
    f.write(s)
driver.close()

In [ ]:
for url in open("list_of_urls.txt").readlines():
    driver.get(url)
    list = driver.find_elements_by_css_selector('h3 a')
    for a in list
        print(a)
        print(a.get_attribute('href'))
        break
    sleep_for = randint(300,2500) / 1000.0
    sleep( sleep_for )
driver.close()

In [ ]:
# Example Create and Import Json 
import json
myJson = {
    "question": "first question",
    "author": 
    {
        "auto": "auto1",
        "name": "nazmul hossain"
    }
}
s = json.dumps(myJson)
with open("myJson.json", "w") as f:
    f.write(s)

In [ ]:
# Example Url traverse by page number 
pages = 10

for page in range(1,pages):
# for url in open("list_of_urls.txt").readlines():
#     driver.get(url)

    url = "http://quotes.toscrape.com/js/page/" + str(page) + "/"
    driver.get(url)
    items = len(driver.find_elements_by_class_name("quote"))
    total = []
    for item in range(items):
        quotes = driver.find_elements_by_class_name("quote")
        for quote in quotes:
            quote_text = quote.find_element_by_class_name('text').text
            author = quote.find_element_by_class_name('author').text
            new = ((quote_text,author))
            total.append(new)
    df = pd.DataFrame(total,columns=['quote','author'])
    df.to_csv('quoted.csv')
    sleep_for = randint(300,2500) / 1000.0
    sleep( sleep_for )
driver.close()